In [5]:
import ROOT
import numpy as np
import math
from ROOT import TMath
import math
from math import exp
import os

In [6]:
%jsroot on

In [7]:
c_sig=ROOT.TChain("mini")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_A.GamGam.root")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_B.GamGam.root")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_C.GamGam.root")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_D.GamGam.root")
c_sig.GetEntries()

7798424

In [8]:
c_ttH=ROOT.TChain("mini")
c_ttH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_341081.ttH125_gamgam.GamGam.root")
c_ggH=ROOT.TChain("mini")
c_ggH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_343981.ggH125_gamgam.GamGam.root")
c_VBFH=ROOT.TChain("mini")
c_VBFH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_345041.VBFH125_gamgam.GamGam.root")
c_WpH=ROOT.TChain("mini")
c_WpH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_345318.WpH125J_Wincl_gamgam.GamGam.root")
c_ZH=ROOT.TChain("mini")
c_ZH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_345319.ZH125J_Zincl_gamgam.GamGam.root")
print(c_ttH.GetEntries(), c_ggH.GetEntries(), c_VBFH.GetEntries(), c_WpH.GetEntries(), c_ZH.GetEntries())

576491 1054711 497468 113765 230900


In [9]:
class Analysis:
    def __init__(self, tree, option):
        self.tree = tree
        self.option = option
        self.accumulated_entries = 0
        

    def begin(self):
        nentries = self.tree.GetEntries()
        print("Processing ",nentries," entries in the "+self.option+" chain")
        self.h_yymass = ROOT.TH1F("h_yymass_"+self.option,"Invariant mass of photon pairs",30,105,160)
    
    def process(self):
        for event in self.tree:
            self.accumulated_entries += 1
            
            if (self.tree.trigP):
                nphotons = 0
                p1 = ROOT.TLorentzVector()
                p2 = ROOT.TLorentzVector()
                leading = ROOT.TLorentzVector()
                subleading = ROOT.TLorentzVector()
                diphotonvector = ROOT.TLorentzVector()
                myy = 0

                for iphoton in range(self.tree.photon_n):
                    pt = self.tree.photon_pt[iphoton]
                    eta = self.tree.photon_eta[iphoton]
                    ptcone30 = self.tree.photon_ptcone30[iphoton]
                    etcone20 = self.tree.photon_etcone20[iphoton]
                    
                    if (self.tree.photon_isTightID[iphoton]):
                        if (((pt/1000>30) and (TMath.Abs(eta)<2.37)) and ((TMath.Abs(eta)<1.37) or (TMath.Abs(eta))>1.52)):
                            if ((ptcone30/pt)<.065) and ((etcone20/pt)<.065):
                                nphotons += 1


                if (nphotons == 2):
                    counter = 0
                    for iphoton in range(self.tree.photon_n):
                        pt = self.tree.photon_pt[iphoton]
                        eta = self.tree.photon_eta[iphoton]
                        phi = self.tree.photon_phi[iphoton]
                        ptcone30 = self.tree.photon_ptcone30[iphoton]
                        etcone20 = self.tree.photon_etcone20[iphoton]
                        E = self.tree.photon_E[iphoton]

                        if (self.tree.photon_isTightID[iphoton]):
                            if (((pt/1000>30) and (TMath.Abs(eta)<2.37)) and ((TMath.Abs(eta)<1.37) or (TMath.Abs(eta))>1.52)):
                                if ((ptcone30/pt)<.065) and ((etcone20/pt)<.065):
                                    if counter == 0:
                                        counter += 1
                                        p1.SetPtEtaPhiE (pt, eta, phi, E)
                                    if counter != 0:
                                        p2.SetPtEtaPhiE (pt, eta, phi, E)
                    
                    if p1.Pt()>p2.Pt():
                        leading = p1
                        subleading = p2
                    if p2.Pt()>p1.Pt():
                        leading = p2
                        subleading = p1



                    if (leading.Pt()/1000) > 40 and (subleading.Pt()/1000) > 30:
                        diphotonvector = leading + subleading
                        myy = diphotonvector.M()/1000

                        if (((leading.Pt()/1000)/myy)>.40) and (((subleading.Pt()/1000)/myy)>.30):

                            if myy > 105 and myy < 160:
                                if self.option == "sig":
                                    weight = 1
                                if self.option != "sig":
                                    weight = (self.tree.mcWeight * self.tree.scaleFactor_PILEUP *\
                                              self.tree.scaleFactor_PhotonTRIGGER * self.tree.scaleFactor_PHOTON)
                                self.h_yymass.Fill(myy, weight)


            if (self.accumulated_entries%10000 == 1):
                print("Processed: ",self.accumulated_entries)
    def terminate(self):
        
        output_name = "output/MYPhotonAnalysis_TIGHT_"+self.option+".root"
        print("Saving to file "+output_name)
        file = ROOT.TFile(output_name, "RECREATE")
        self.h_yymass.Write()
        file.Close()
        print("Done saving!")

In [10]:
a_sig=Analysis(c_sig,"sig")
a_sig.begin()
a_sig.process()
a_sig.terminate()

a_ttH=Analysis(c_ttH,"ttH")
a_ttH.begin()
a_ttH.process()
a_ttH.terminate()

a_VBFH=Analysis(c_VBFH,"VBFH")
a_VBFH.begin()
a_VBFH.process()
a_VBFH.terminate()

a_ggH=Analysis(c_ggH,"ggH")
a_ggH.begin()
a_ggH.process()
a_ggH.terminate()

a_ZH=Analysis(c_ZH,"ZH")
a_ZH.begin()
a_ZH.process()
a_ZH.terminate()

a_WpH=Analysis(c_WpH,"WpH")
a_WpH.begin()
a_WpH.process()
a_WpH.terminate()

Processing  7798424  entries in the sig chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Processed:  120001
Processed:  130001
Processed:  140001
Processed:  150001
Processed:  160001
Processed:  170001
Processed:  180001
Processed:  190001
Processed:  200001
Processed:  210001
Processed:  220001
Processed:  230001
Processed:  240001
Processed:  250001
Processed:  260001
Processed:  270001
Processed:  280001
Processed:  290001
Processed:  300001
Processed:  310001
Processed:  320001
Processed:  330001
Processed:  340001
Processed:  350001
Processed:  360001
Processed:  370001
Processed:  380001
Processed:  390001
Processed:  400001
Processed:  410001
Processed:  420001
Processed:  430001
Processed:  440001
Processed:  450001
Processed:  460001
Processed:  470001
Processed:  480001
Processed:  490001
Processed:  500001

Processed:  4140001
Processed:  4150001
Processed:  4160001
Processed:  4170001
Processed:  4180001
Processed:  4190001
Processed:  4200001
Processed:  4210001
Processed:  4220001
Processed:  4230001
Processed:  4240001
Processed:  4250001
Processed:  4260001
Processed:  4270001
Processed:  4280001
Processed:  4290001
Processed:  4300001
Processed:  4310001
Processed:  4320001
Processed:  4330001
Processed:  4340001
Processed:  4350001
Processed:  4360001
Processed:  4370001
Processed:  4380001
Processed:  4390001
Processed:  4400001
Processed:  4410001
Processed:  4420001
Processed:  4430001
Processed:  4440001
Processed:  4450001
Processed:  4460001
Processed:  4470001
Processed:  4480001
Processed:  4490001
Processed:  4500001
Processed:  4510001
Processed:  4520001
Processed:  4530001
Processed:  4540001
Processed:  4550001
Processed:  4560001
Processed:  4570001
Processed:  4580001
Processed:  4590001
Processed:  4600001
Processed:  4610001
Processed:  4620001
Processed:  4630001


Processed:  410001
Processed:  420001
Processed:  430001
Processed:  440001
Processed:  450001
Processed:  460001
Processed:  470001
Processed:  480001
Processed:  490001
Processed:  500001
Processed:  510001
Processed:  520001
Processed:  530001
Processed:  540001
Processed:  550001
Processed:  560001
Processed:  570001
Saving to file output/MYPhotonAnalysis_TIGHT_ttH.root
Done saving!
Processing  497468  entries in the VBFH chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Processed:  120001
Processed:  130001
Processed:  140001
Processed:  150001
Processed:  160001
Processed:  170001
Processed:  180001
Processed:  190001
Processed:  200001
Processed:  210001
Processed:  220001
Processed:  230001
Processed:  240001
Processed:  250001
Processed:  260001
Processed:  270001
Processed:  280001
Processed:  290001
Processe

In [11]:
canvases = {}
files = {}
histos = {}
legends={}
canvases2 = {}
files2 = {}
histos2 = {}
legends2={}
#
def Plotter1():
    # Tags for plots and samples
    plots = ["yymass"]
    samples = ["sig","ggH","ttH","VBFH","WpH","ZH"]
    
    #scaling and colors
    lum = 10064
    scalefactors = {"sig":1,"ttH":(lum*.0000026433864/55922617.6297),"ggH":(lum*.102/55922617.6297),\
                    "VBFH":(lum*.008518764/3441426.13711),"WpH":(lum*.0019654512/213799.958463),"ZH":(lum*.0017347836/358401.082034)}
    fillColors = {"sig":2, "ttH":4, "ggH":4, "VBFH":4, "WpH":4, "ZH":4}
    fillStyles = {"sig":3003, "ttH":3003, "ggH":3003, "VBFH":3003, "WpH":3003, "ZH":3003}
    xLabels = {"yymass":"diphoton invariant mass"}
    yLabels = {"yymass":"Number/bin"}
    legendLabels = {"sig":"Signal (H #rightarrow yy)", "ttH":"ttH", "ggH":"ggH", "VBFH":"VBFH", "WpH":"WpH", "ZH":"ZH"}
    # Read in histograms, manipulate settings
    for tag in samples:
        filename = "output/MYPhotonAnalysis_TIGHT_"+tag+".root"
        print("Opening "+filename)
        files[tag] = ROOT.TFile(filename,"READ")
        for h in plots:
            legends[h+"_"+tag] = ROOT.TLegend(0.5, 0.7, 0.9, 0.9)
            histo=files[tag].Get("h_"+h+"_"+tag)
            histo.SetFillColor(fillColors[tag])
            histo.SetLineColor(fillColors[tag])
            histo.SetFillStyle(fillStyles[tag])
            histo.Scale(scalefactors[tag])
            histo.GetXaxis().SetTitle(xLabels[h])
            histo.GetYaxis().SetTitle(yLabels[h])
            histo.SetStats(0)
            histos[h+"_"+tag] = histo
            canvases[h+"_"+tag] = ROOT.TCanvas(h+"_"+tag,"Canvas for "+h+"_"+tag, 800, 600)
            legends[h+"_"+tag].AddEntry(histos[h+"_"+tag],legendLabels[tag],"f")
            histos[h+"_"+tag].Draw()
            legends[h+"_"+tag].Draw()
            
            

    #overlays
    for h in plots:
        legends2[h] = ROOT.TLegend(0.5, 0.7, 0.9, 0.9)
        canvases2[h] = ROOT.TCanvas(h,"Canvas for "+h, 800, 600)
        for tag in samples:
            if tag != "sig" and tag != "ggH":
                #legends2[h].AddEntry(histos[h+"_"+tag],legendLabels[tag],"f")
                histos[h+"_ggH"].Add(histos[h+"_"+tag])
        
        legends2[h].AddEntry("Sum of hhG+ttH+VBFH+WpH+ZH")
        legends2[h].Draw()
        histos[h+"_ggH"].SetFillStyle(2)
        histos[h+"_ggH"].SetLineStyle(1)
        histos[h+"_ggH"].SetLineWidth(2)
        histos[h+"_ggH"].SetLineColor(ROOT.kBlack)
        histos[h+"_ggH"].Draw()
        legends2[h].Draw()

In [12]:
Plotter1()

Opening output/MYPhotonAnalysis_TIGHT_sig.root
Opening output/MYPhotonAnalysis_TIGHT_ggH.root
Opening output/MYPhotonAnalysis_TIGHT_ttH.root
Opening output/MYPhotonAnalysis_TIGHT_VBFH.root
Opening output/MYPhotonAnalysis_TIGHT_WpH.root
Opening output/MYPhotonAnalysis_TIGHT_ZH.root


In [13]:
sig2 = histos["yymass_sig"]

In [14]:
polyfit2 = ROOT.TF1("polyfit2", "([0] + [1]*x + [2]*x**2 + [3]*x**3 + [4]*exp((-.5*((x-[5])/[6])**2)))", 105, 160)
polyfit2.SetParameter(5, 143)
polyfit2.SetParLimits(5, 140, 150)
polyfit2.SetParameter(6, 2.39)
polyfit2.SetParLimits(6, 1, 3)
polyfit2.SetLineColor(2)
polyfit2.SetLineStyle(1)
polyfit2.SetLineWidth(2)
sig2.Fit("polyfit2", "S", "E SAME", 105, 160)

datagauss2 = ROOT.TF1("datagauss2", "89.8905*exp((-.5*((x-143.025)/1.81455)**2))", 105, 160)

 FCN=25.78 FROM MIGRAD    STATUS=CONVERGED    1433 CALLS        1434 TOTAL
                     EDM=9.56893e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.8 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           6.25992e+04   6.64698e+03  -8.51019e+00  -6.35526e-05
   2  p1          -1.13576e+03   1.51354e+02   1.90701e-01  -9.46419e-03
   3  p2           7.12835e+00   1.14015e+00  -1.41314e-03  -1.39449e+00
   4  p3          -1.52882e-02   2.84124e-03   3.46830e-06  -2.04435e+02
   5  p4           8.34813e+01   3.35052e+01   1.03732e-01  -1.81941e-05
   6  p5           1.43301e+02   9.45516e-01  -5.55080e-04  -9.16134e-04
   7  p6           2.10942e+00   6.04332e-01  -2.34614e-03  -1.63406e-03


In [15]:
ROOT.gROOT.SetStyle("ATLAS")
fhisto = ROOT.TCanvas("fhisto", "", 700, 750)
mainplot = ROOT.TPad("mainplot","",0,0.35,1,1)
ratioplot = ROOT.TPad("ratioplot","", 0, 0, 1, .35)
mainplot.SetTickx(False)
mainplot.SetTicky(False)
mainplot.SetLeftMargin(.14)
mainplot.SetRightMargin(.05)
mainplot.SetBottomMargin(0)
ratioplot.SetTickx(False)
ratioplot.SetTicky(False)
ratioplot.SetLeftMargin(.14)
ratioplot.SetRightMargin(.05)
ratioplot.SetTopMargin(0)
ratioplot.SetBottomMargin(0.3)
mainplot.Draw()
ratioplot.Draw()
fhisto.Draw()




mainplot.cd()

histos["yymass_sig"].SetMarkerStyle(20)
histos["yymass_sig"].SetMarkerSize(.5)
histos["yymass_sig"].SetLineWidth(1)
histos["yymass_sig"].SetLineColor(ROOT.kBlack)
histos["yymass_sig"].SetMinimum(1e-3)
histos["yymass_sig"].SetMaximum(6e3)
histos["yymass_sig"].GetYaxis().SetLabelSize(0.045)
histos["yymass_sig"].GetYaxis().SetTitleSize(0.05)
histos["yymass_sig"].SetStats(0)
histos["yymass_sig"].SetTitle("Higgs peak")


polygaus = ROOT.TF1("polygaus", "([0] + [1]*x + [2]*x**2 + [3]*x**3 + [4]*exp((-.5*((x-[5])/[6])**2)))", 105, 160)
polygaus.SetParameter(4, 119.1)
polygaus.SetParLimits(4, 110, 1000)
polygaus.SetParameter(5, 125)
polygaus.SetParLimits(5, 120, 130)
polygaus.SetParameter(6, 2.39)
polygaus.SetParLimits(6, 2, 2.5)
polygaus.SetLineColor(ROOT.kRed)
polygaus.SetLineStyle(1)
polygaus.SetLineWidth(2)
histos["yymass_sig"].Fit("polygaus", "", "E SAME", 105, 160)
polygaus.Draw("SAME")


poly = ROOT.TF1("poly", "([0] + [1]*x + [2]*x**2 + [3]*x**3)", 105, 160)
for i in range (0,3):
    poly.FixParameter(i, polygaus.GetParameter(i))
poly.SetLineColor(4)
poly.SetLineStyle(2)
poly.SetLineWidth(2)
histos["yymass_sig"].Fit("poly", "", "E SAME", 105, 160)
poly.Draw("SAME")

histos["yymass_ggH"].SetLineColor(ROOT.kBlack)
histos["yymass_ggH"].SetLineWidth(2)
histos["yymass_ggH"].SetLineStyle(1)
histos["yymass_ggH"].Draw("SAME")


legend = ROOT.TLegend(0.7, 0.65, .9, .9)
legend.SetTextFont(42)
legend.SetBorderSize(0)
legend.SetTextSize(0.1)
legend.AddEntry(histos["yymass_sig"], "Data" ,"lep")
legend.AddEntry(polygaus, "Signal + Bkg.", "l")
legend.AddEntry(poly, "Background", "l")
legend.AddEntry(histos["yymass_ggH"], "Signal", "l")
legend.Draw("SAME")


ratioplot.cd()

ratio = ROOT.TH1F("ratio","",10000,105,160)
ratio.SetMinimum(-125)
ratio.SetMaximum(250)
ratio.GetXaxis().SetLabelSize(0.08)
ratio.GetXaxis().SetTitleSize(0.12)
ratio.GetXaxis().SetTitleOffset(1.0)
ratio.GetYaxis().SetLabelSize(0.07)
ratio.GetYaxis().SetTitleSize(0.09)
ratio.GetYaxis().SetTitle("Data - Bkg.")
ratio.GetYaxis().CenterTitle()
ratio.GetYaxis().SetTitleOffset(0.7)
ratio.GetYaxis().SetNdivisions(503, False)
ratio.GetYaxis().ChangeLabel(-1, -1, 0)
ratio.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
ratio.Draw()


zero = ROOT.TF1("zero", "0", 105, 160)
zero.SetLineStyle(2)
zero.SetLineWidth(1)
zero.SetLineColor(4)
zero.Draw("SAME")

datagauss = ROOT.TF1("datagauss", "136.626*exp((-.5*((x-124.759)/2.27372)**2))+89.8905*exp((-.5*((x-143.025)/1.81455)**2))", 105, 160)
datagauss.SetLineStyle(1)
datagauss.SetLineWidth(2)
datagauss.SetLineColor(2)
datagauss.Draw("SAME")
#ratiosig = ROOT.TH1F("ratiosig", "ratiosig", 10000, 105, 160)
#ratiosig.Eval(polygaus)
#ratiosig.SetLineColor(2)
#ratiosig.SetLineStyle(1)
#ratiosig.SetLineWidth(2)
#ratiosig.Add(poly, -1)
#ratiosig.Draw("SAME")

#datagauss2 = ROOT.TF1("datagauss2", "89.8905*exp((-.5*((x-143.025)/1.81455)**2))", 105, 160)
#datagauss2.SetLineStyle(1)
#datagauss2.SetLineWidth(2)
#datagauss2.SetLineColor(2)
#datagauss2.Draw("SAME")

dataerror = histos["yymass_sig"].Clone()
dataerror.Add(poly, -1)
for i in range(1, histos["yymass_sig"].GetNbinsX()):
    dataerror.SetBinError(i, histos["yymass_sig"].GetBinError(i))
dataerror.Draw("E SAME")

 FCN=145.535 FROM MIGRAD    STATUS=FAILED       1034 CALLS        1035 TOTAL
                     EDM=0.0272669    STRATEGY= 1      ERR MATRIX NOT POS-DEF
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.26624e+04   6.32739e+01   4.25919e-02  -3.66323e-03
   2  p1          -1.26403e+01   6.82948e-01   3.02908e-04   4.63143e-03
   3  p2          -1.22549e+00   4.70331e-03   2.11111e-06   6.66054e-01
   4  p3           5.27106e-03   2.34051e-05   1.44742e-08   1.25088e+02
   5  p4           1.10000e+02   3.61776e+00   7.55732e-04** at limit **
   6  p5           1.25042e+02   1.23365e+00   1.47420e-03   4.68623e-05
   7  p6           2.00000e+00   7.66035e-02   4.76386e-03** at limit **
 FCN=125.51 FROM MIGRAD    STATUS=CONVERGED      14 CALLS          15 TOTAL
                     EDM=3.28074e-22    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                   

Warning in <Fit>: Abnormal termination of minimization.


In [16]:
gauss = ROOT.TF1("gaus", "([0]*exp((-.5*((x-[1])/[2])**2)))", 105, 160)
gauss.SetParameter(0, 119.1)
gauss.SetParLimits(0, 110, 130)
gauss.SetParameter(1, 125)
gauss.SetParLimits(1, 120, 130)
gauss.SetParameter(2, 2.39)
gauss.SetParLimits(2, 2, 2.5)
histos["yymass_ggH"].Fit("gaus")




HiggseventsvsMC = ROOT.TCanvas("Higgs events","Higgs events", 600, 600)


Higgs = ROOT.TH1F("Higgs", "Higgs events", 1000, 105, 160)
Higgs.Eval(polygaus)
Higgs.SetLineColor(2)
Higgs.SetLineStyle(1)
Higgs.SetLineWidth(2)
Higgs.Add(poly, -1)
Higgs.SetMinimum(-125)
Higgs.SetMaximum(250)
Higgs.Draw()
gauss.SetLineStyle(2)
gauss.SetLineColor(1)
gauss.SetLineWidth(3)
gauss.Draw("SAME")

dataerror2 = histos["yymass_sig"].Clone()
dataerror2.Add(poly, -1)
for i in range(1, histos["yymass_sig"].GetNbinsX()):
    dataerror2.SetBinError(i, histos["yymass_sig"].GetBinError(i))
dataerror2.Draw("E SAME")

legendMCvsdata = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
legendMCvsdata.SetTextFont(42)
legendMCvsdata.SetBorderSize(0)
legendMCvsdata.SetTextSize(0.7)
legendMCvsdata.AddEntry(gauss, "MC" ,"l")
legendMCvsdata.AddEntry(Higgs, "Data", "l")

HiggseventsvsMC.Draw()
legendMCvsdata.Draw()

 FCN=10725.2 FROM MIGRAD    STATUS=CONVERGED      77 CALLS          78 TOTAL
                     EDM=7.69541e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.10000e+02   1.72721e-03   9.39774e-04** at limit **
   2  p1           1.24938e+02   3.02624e-03   3.05973e-05  -1.27877e+00
   3  p2           2.39868e+00   2.02103e-03   5.08408e-04  -7.25760e-02


In [17]:
Higgsgaussfit1 = ROOT.TF1("Higgsgaussfit1", "[0]*exp((-.5*((x-[1])/[2])**2))", 105, 160)
Higgsgaussfit1.SetParameter(0, 119.1)
Higgsgaussfit1.SetParLimits(0, 110, 1000)
Higgsgaussfit1.SetParameter(1, 125)
Higgsgaussfit1.SetParLimits(1, 120, 130)
Higgsgaussfit1.SetParameter(2, 2.39)
Higgsgaussfit1.SetParLimits(2, 2, 2.5)
Higgsevents1 = histos["yymass_sig"].Clone()
Higgsevents1.Add(poly, -1)
Higgsevents1.Fit("Higgsgaussfit1", "S", "E SAME", 115, 135)

polychifit = ROOT.TF1("polychifit", "([0] + [1]*x + [2]*x**2 + [3]*x**3)", 105, 160)
Higgsevents2 = histos["yymass_sig"].Clone()
Higgsevents2.Fit("polychifit", "S", "E SAME", 105, 160)


Higgsevents3 = histos["yymass_sig"].Clone()
Higgsevents3.Add(poly, -1)
Higgsnullfit = ROOT.TF1("Higgsnullfit", "[0]", 105, 160)
Higgsnullfit.SetParameter(0, 0)
Higgsnullfit.SetParLimits(0, -.1, .1)
Higgsevents3.Fit("Higgsnullfit", "S", "E SAME", 115, 135)


Higgsgaussfit4 = ROOT.TF1("Higgsgaussfit4", "[0]*exp((-.5*((x-[1])/[2])**2))", 105, 160)
Higgsgaussfit4.SetParameter(1, 143)
Higgsgaussfit4.SetParLimits(1, 140, 150)
Higgsgaussfit4.SetParameter(2, 2.39)
Higgsgaussfit4.SetParLimits(2, 0, 5)
Higgsevents4 = histos["yymass_sig"].Clone()
Higgsevents4.Add(poly, -1)
Higgsevents4.Fit("Higgsgaussfit4", "S", "E SAME", 132, 152)



test1 = ROOT.TCanvas("test1","MC vs data", 600, 600)
Higgsevents1.Draw()
Higgsgaussfit4.Draw()

test1.Draw()


Higgspeakevents = (Higgsgaussfit1.Integral(115,135) / (55/30))
Secondpeakevents = (Higgsgaussfit4.Integral(132,152)/(55/30))



print("Total events above background in Higgs peak:")
print(Higgspeakevents)
print("Total events above background in 2nd peak peak:")
print(Secondpeakevents)
print("chi^2 for Higgs peak fit")
print(Higgsgaussfit1.GetChisquare())
print("NDF for Higgs peak fit")
print(Higgsgaussfit1.GetNDF())
print("chi^2 for poly fit")
print(polychifit.GetChisquare())
print("NDF for poly fit")
print(polychifit.GetNDF())
print("chi^2 for best fit, polygaus")
print(polygaus.GetChisquare())
print("NDF for best fit, polygaus")
print(polygaus.GetNDF())
print("chi^2 for null hypothesis")
print(Higgsnullfit.GetChisquare())
print("NDF for null hypothesis")
print(Higgsnullfit.GetNDF())
print("chi^2 for 2nd peak gauss fit")
print(Higgsgaussfit4.GetChisquare())
print("NDF 2nd peak gauss fit")
print(Higgsgaussfit4.GetNDF())

Total events above background in Higgs peak:
300.7952226924212
Total events above background in 2nd peak peak:
428.44472537619095
chi^2 for Higgs peak fit
41.71804431624439
NDF for Higgs peak fit
8
chi^2 for poly fit
32.21071965839485
NDF for poly fit
26
chi^2 for best fit, polygaus
145.5347390895401
NDF for best fit, polygaus
23
chi^2 for null hypothesis
21.201388568544193
NDF for null hypothesis
10
chi^2 for 2nd peak gauss fit
10.185946926870033
NDF 2nd peak gauss fit
8
 FCN=41.718 FROM MIGRAD    STATUS=CONVERGED      56 CALLS          57 TOTAL
                     EDM=3.77681e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.10000e+02   3.53200e+00   4.02144e-04** at limit **
   2  p1           1.25005e+02   1.21142e+00   7.79537e-04   1.56540e-04
   3  p2           2.00000e+00   7.38728e-02   2.51639e-03** at limit **
 FCN